# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
#載入套件
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
#指定網址
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
#使用webdriver.Chrime()建立Google Chrime瀏覽器物件
browser = webdriver.Chrome()
#以get()方法連結到指定網址
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#通過左邊控制橫向和縱向滾動條scrollTo(x, y）
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    time.sleep(2) #是Python程序暫停執行2秒
    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print('到達頁面底端')
        break
    print('網頁更新中...')
    last_height = new_height

網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    # 使用 GET 方式下載目標網頁
    resp = requests.get(link_url)
    #轉換編碼至UTF-8
    resp.encoding = 'utf-8'
    #print(resp.text)
    # 以 BeautifulSoup 解析 HTML 原始碼
    soup = BeautifulSoup(resp.text, 'lxml') #指定html.parser作為解析器
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
# 以 BeautifulSoup 解析 HTML 原始碼
soup = BeautifulSoup(html, "lxml") #指定html.parser作為解析器
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


'''
補充說明:
enumerate多用於在for循環中得到計數，利用它可以同時獲得索引和值，即需要index和value值的時候可以使用enumerate。
'''

----------------------------------------------------------------------

[0] 5G標金創全球天價！「你不退我不退」原因曝　官員無奈揭內幕

圖、文／鏡週刊
去年底政府釋出5G頻譜，主力頻段3.5GHz僅釋出270MHz頻寬，電信五雄為搶下頻寬，贏得這場生存之戰，莫不殺紅眼，截至週一（元月6日），3.5GHz頻段標金已達988.47億元，每單位（10MHz）標金高達36.61億元，刷新全球紀錄。
 
上週四（元月2日）新年伊始，各界正在喜迎新的一年到來，對電信五雄來說，卻是惡夢的開始，「每天都抱著期望去競標5G，卻都空手而歸，眼看標金越來越高，真不知道哪天才能結束？」一名電信三強高層無奈的說。
▲為搶5G執照台灣五家電信業者殺紅眼，標金也創下全球新高。（圖／鏡週刊提供，下同）
去年底政府釋出5G頻譜，共釋出3.5GHz、28GHz與1800MHz 3個頻段，最熱門的3.5GHz頻段，截至週一標金已達988.47億元，才短短19天，眼看競標金額就要扣關千億，中華電信、台灣大哥大、遠傳電信、亞太電信和台灣之星五家電信業者仍沒有鬆手跡象，讓這場競標戰宛如「失速列車」一般失控。
本刊調查，中華電信、台灣大哥大、遠傳電信三強互不相讓，堅持各自拿下100MHz頻寬，台灣之星和亞太電信兩雄各盤算拿40和20MHz，總計5家電信業者需求就要360MHz頻寬，遠超過政府釋出的270MHz頻寬，中華電信董事長謝繼茂就坦言，這次頻寬的需求大於供給。
知情人士透露，NCC其實曾找過專業顧問評估，以台灣電信市場的競爭，最適合家數約3.3家，也就是說隨市場競爭加劇，5G營運後，市場也會出現收斂，意味著有業者可能會退場，「這次頻譜競爭如此激烈，不惜重金都要搶標，就因業者視為生存之戰。」市場人士分析。
 
由於中華電信、台灣大哥大、遠傳電信三強互不相讓，堅持各自拿下100MHz頻寬，台灣之星和亞太電信兩雄各盤算拿40和20MHz，總計5家電信業者需求就要360MHz頻寬，遠超過政府釋出的270MHz頻寬，中華電信董事長謝繼茂就坦言，這次頻寬的需求大於供給。
眼見5家電信業者僵持不下，去年耶誕節後、5G競標戰邁入第12天，每單位（10MHz）標金已達24.15億元，電信獲利王、台灣大哥大董事長蔡明忠眼見這場戰爭短期難以結束，竟率

▲隨著中美貿易戰可望完成第一階段協議簽署，以及5G等新興商機的需求，今年首季可望迎春。（圖／記者徐文彬攝）
記者吳靜君／台北報導
財政部今（12）日公布海關進出口，去（2019）年12月的出口金額為295.0億美元，創下歷史單月次高，年增率4.0%。財政部統計處處長蔡美娜表示，主計處預估今年首季的出口成長率達4.1%%，台灣的出口已經走出這波低潮。
去年12月進出口　雙雙有好成績
但今年因為農曆年正好落於元月，過往春節年假落在元月很少，使得元月的公天數只剩下17天，因為這個效應所以預期1月的年增率可能為-4.5%到-6.5%，但是不用太負面解讀。
去年第4季的表現，優於主計處預期，在金額方面多了7.2億美元，年增率也增加了0.9個百分點。
根據財政部所公布的數字，去年12月出口金額達295.0億美元、第4季的出口金額達到870.6億美元，創歷史單季新高；另外去年全年的出口金額也有3293.4億美元，年減率為1.4%。
進口也有很好的表現，去年12月的進口金額為269.9億美元，為歷年單月次高，較11月增加11.1%，較前一年同期增加13.9%。第4季進口金額為763億美元，較前一年同季增加4.9%。全年進口2858.6億美元，較前一年增加0.4%。
出口貨品中，資通訊產品及電腦零組件及其下的積體電路都相當有表現，12月電子零組件的年增率15.1%、積體電路也有11.9%，單月規模也都創歷史新高。
3大原因　第一季出口走向光明
蔡美娜分析，展望今（2020）年第一季，有利的原因在於：一，中美貿易協商短期內可望簽署第一階段協議，雙方關係有改善的契機，國際經濟面臨的不確定性逐漸淡化，有助於終端需求和原物料行情。
二，以及5G通訊、人工智慧等新興商機（AI）備受看好，台灣半導體產業設備進口連兩個月呈現年成長率都1.6倍，顯示廠商看好未來發展，電子產業未來商機可期。
三，全球供應鏈加速重組，廠商調升在台的產能配置。前述的原因之下，第一季出口可望穩健成長。
----------------------------------------------------------------------

[10] 美乳業臨寒冬！煉乳發明商、百年企業Borden聲請破產

▲博登乳業聲請破產。（圖／達志影像／美聯社）
記者王曉敏／綜合報導
美國歷史最悠久、規模最大的乳製品公司之一博登乳

▲ 美盛資產管理旗下團隊。（圖／記者李瑞瑾攝）
記者李瑞瑾／台北報導
2020年才剛開始，就遇到了美伊衝突導致市場出現波動，美盛資產管理邀請旗下布蘭迪全球投資管理亞洲執行長Tariq Ahmad、凱利投資董事、客戶投資組合經理Kathleen Anderson、睿安基礎建設資深投資組合經理Charles Hamieh，以及西方資產管理亞洲投資主管、投資組合經理孫應梅四人來台，分享對今年全球經濟局勢的看法。
2019年，聯準會跟各國央行鬆綁貨幣政策，接下來，這些刺激政策都是支撐全球經濟的主因。而這些貨幣政策與財政政策，在2020年也是支撐全球經濟發展的正向要素。
低通膨環境延續，已開發與新興市場通膨都低於目標，因此各國都還是有降息空間。但貨幣政策發揮空間有限，不少國家開始靠財政政策刺激，歐洲目前做最少，德國汽車製造業在2019年表現疲弱，雖然已經觸底反彈，但仍需靠財政政策支撐。而各國紛紛推出的刺激政策，也讓不少投資機會顯現，景氣循環股表現可望領先指數。
地緣政治緊張升溫，但還是有一些資產類別表現很好。美股在大選當年有74%的機率上漲，雖被指出評價偏高，但因低利率環境持續與景氣回升，仍可以在各產業找到投資機會，其中投資評價相對較低者，包含能源、金融、醫療服務、科技硬體等產業，投資機會顯現。
他們認為，2020上半年看好各資產貨幣條件，預期資金會出走美國到其他國家，而下半年美國要選舉，將會出現更多不確定性。
在新興市場部分，2020年表現也將優於成熟市場，並且會擴大差距。他們認為，其實有些新興經濟體債務負擔比某些已開發國家輕，很多國家的建設也都比已開發國家好了，成長速度相當快速；但也提醒投資人，成長速度快不代表報酬會好，雖然看好新興市場債市會表現不錯，但股市仍要慎選標的。
從總經角度來看，全球經濟實現軟著陸，今年也會持續成長，只是成長會放緩。財政政策與貨幣政策會持續支稱2020年市場信心，不少亞洲國家、新興市場、歐洲跟中國大陸，都有很好的投資機會。
----------------------------------------------------------------------

[22] 傳因蘋果取消印度千億投資案　鴻海聲明：錯誤不實說法

▲鴻海土城總部。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
鴻海（2317）今（7）日發出聲明，針對有關媒體指稱鴻海因為

In [6]:
# 關閉瀏覽器
browser.quit();